In [3]:
import mlflow
import mlflow.keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import pandas as pd
import joblib
from tensorflow.keras.models import save_model

# Cargar los datos desde el archivo CSV
datos = pd.read_csv('datossaber11_original.csv')

# Lista de columnas seleccionadas
columnas_seleccionadas = ['cole_area_ubicacion', 'cole_bilingue', 'cole_calendario',
                          'cole_genero', 'estu_depto_presentacion',
                          'fami_estratovivienda', 'fami_tieneautomovil', 'fami_tienecomputador',
                          'fami_tieneinternet', 'fami_tienelavadora', 'desemp_ingles', 'punt_global']

# Crear un nuevo DataFrame con las columnas seleccionadas
data = datos[columnas_seleccionadas]

# Selecciona las características (X) y la variable objetivo (y)
X = data.drop('punt_global', axis=1)  # Excluye la variable objetivo
y = data['punt_global']

# Normalizar la variable objetivo
scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1)).flatten()

# Divide los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y_scaled, test_size=0.2, random_state=42)

# Definir las columnas categóricas
categorical_features = ['cole_area_ubicacion', 'cole_bilingue', 'cole_calendario',
                        'cole_genero', 'estu_depto_presentacion',
                        'fami_estratovivienda', 'fami_tieneautomovil', 'fami_tienecomputador',
                        'fami_tieneinternet', 'fami_tienelavadora', 'desemp_ingles']

# Crear el transformador de columnas
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Crear el pipeline de preprocesamiento
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('scaler', StandardScaler())])

# Aplicar el pipeline a los datos de entrenamiento y prueba
X_train_scaled = pipeline.fit_transform(X_train)
X_test_scaled = pipeline.transform(X_test)

# Parámetros variables
epochs = 20
batch_size = 32

# Configuración y entrenamiento del modelo con MLflow
# Crear y compilar el modelo
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1)  # Capa de salida sin activación para regresión
])
    
model.compile(optimizer='adam', loss='mean_squared_error')
    
# Entrenar el modelo
history = model.fit(X_train_scaled, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)

# Evaluar el modelo en el conjunto de prueba
y_pred_scaled = model.predict(X_test_scaled).flatten()

# Desnormalizar las predicciones
y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()
y_test_actual = scaler_y.inverse_transform(y_test.reshape(-1, 1)).flatten()

# Calcular métricas
mse = mean_squared_error(y_test_actual, y_pred)
mae = mean_absolute_error(y_test_actual, y_pred)
mre = np.mean(np.abs((y_test_actual - y_pred) / y_test_actual))

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"Mean Relative Error: {mre}")

# Guardar el modelo
save_model(model, 'modelo_saber11.h5')

# Guardar el pipeline y el scaler de y
joblib.dump(pipeline, 'pipeline.pkl')
joblib.dump(scaler_y, 'scaler_y.pkl')

Epoch 1/20


c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9438/9438 ━━━━━━━━━━━━━━━━━━━━ 10s 966us/step - loss: 0.3731 - val_loss: 0.3565
Epoch 2/20
9438/9438 ━━━━━━━━━━━━━━━━━━━━ 9s 923us/step - loss: 0.3590 - val_loss: 0.3561
Epoch 3/20
9438/9438 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - loss: 0.3574 - val_loss: 0.3540
Epoch 4/20
9438/9438 ━━━━━━━━━━━━━━━━━━━━ 9s 947us/step - loss: 0.3581 - val_loss: 0.3539
Epoch 5/20
9438/9438 ━━━━━━━━━━━━━━━━━━━━ 9s 930us/step - loss: 0.3561 - val_loss: 0.3542
Epoch 6/20
9438/9438 ━━━━━━━━━━━━━━━━━━━━ 9s 932us/step - loss: 0.3568 - val_loss: 0.3534
Epoch 7/20
9438/9438 ━━━━━━━━━━━━━━━━━━━━ 9s 934us/step - loss: 0.3552 - val_loss: 0.3540
Epoch 8/20
9438/9438 ━━━━━━━━━━━━━━━━━━━━ 9s 917us/step - loss: 0.3575 - val_loss: 0.3534
Epoch 9/20
9438/9438 ━━━━━━━━━━━━━━━━━━━━ 9s 909us/step - loss: 0.3582 - val_loss: 0.3539
Epoch 10/20
9438/9438 ━━━━━━━━━━━━━━━━━━━━ 9s 906us/step - loss: 0.3560 - val_loss: 0.3542
Epoch 11/20
9438/9438 ━━━━━━━━━━━━━━━━━━━━ 9s 932us/step - loss: 0.3572 - val_loss: 0.3530
Epoch 12/20
9438/94

Mean Squared Error: 973.8039883413746
Mean Absolute Error: 25.016805307019805
Mean Relative Error: 0.1016217534073071


['scaler_y.pkl']

In [7]:
model.save('modelo_saber11.h5')